In [ ]:
import pandas as pd
import sys
import json
import plotly.express as px
import ast

sys.path.insert(0,'../../src')
sys.path.insert(0,'../..')

from consumer import ConsumerFile



In [ ]:
N = [100, 200, 1000]
X = [0.5, 1, 2, 3]
# manjka en senzor za pritisk
data_files = {
    # "B100_hour_SS_input": ["53"],
    "B200_hour_SS_input": ["52", "56"],
    "B300_hour_SS_input": ["51", "57"]
    }
base_data_location = "../../data/JEMS/"

In [ ]:
with open("../../configuration/JEMS/welford.json") as c:
    conf = json.load(c)
    for data_file in data_files:
        sensors = data_files[data_file]
        data_location = base_data_location + data_file + ".csv"
        df = pd.read_csv(data_location) 
        for sensor in sensors:
            # Prepare file
            vals = df[["timestamp", sensor]]
            val_file_name = "data/{}_sensor{}.csv".format(data_file, sensor)
            vals.to_csv(val_file_name, index=False)
            conf["file_name"] = val_file_name

            # Loop through hyperparameters
            for n in N:
                for x in X:
                    
                    # Set configurations
                    conf["anomaly_detection_conf"][0]["N"] = n
                    conf["anomaly_detection_conf"][0]["X"] = x
                    file_name = "{}_sensor{}_welford_N{}_X{}.csv".format(data_file, sensor, n, x)
                    conf["anomaly_detection_conf"][0]["output_conf"][0]["file_name"] = file_name

                    # Run
                    consumer = ConsumerFile(conf)
                    consumer.read()
                    print(file_name)

In [ ]:
res = pd.read_csv("log/sensor53_welford_N1000_X2.csv", index_col="timestamp")

res["status_code"] = res["status_code"].astype(str)
res["value"] = res.value.apply(lambda x: ast.literal_eval(x)[0])

fig = px.scatter(res, y="value", color="status_code", hover_data=['value'], color_discrete_sequence=["blue", "green", "yellow", "red"])
fig.show()